# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [27]:
# import libraries
import pandas as pd
import numpy as np

import re
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sqlalchemy import create_engine

import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('tab_DisasterResponse', engine)

# Define X (Input) and Y (Output)
X = df['message'] # Message as the input variable
y = df.drop(columns=['id', 'message', 'original', 'genre'])

df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1.0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
model = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', RandomForestClassifier())
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# Split into train and test set; chosen random_state ensures deterministic results!
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Train Model
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_pred_test = model.predict(X_test)

# classification report on test data
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.84      0.90      0.87      4938
               request       0.84      0.39      0.53      1104
                 offer       0.00      0.00      0.00        26
           aid_related       0.79      0.41      0.54      2714
          medical_help       0.33      0.01      0.01       542
      medical_products       0.85      0.03      0.06       347
     search_and_rescue       0.75      0.02      0.03       192
              security       1.00      0.01      0.01       133
              military       0.25      0.00      0.01       225
           child_alone       0.00      0.00      0.00         0
                 water       0.86      0.16      0.27       430
                  food       0.84      0.26      0.40       731
               shelter       0.80      0.11      0.20       598
              clothing       1.00      0.03      0.06       101
                 money       0.50      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
# get list of model parameters
model.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7ba311690048>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None, verbose=0,
               warm_

In [8]:
# Set parameters for GridSearch
parameters = {
    'clf__n_estimators': [25, 50],
    'clf__min_samples_split': [2, 3]
}

cv = GridSearchCV(model, param_grid=parameters, verbose=10)
cv.fit(X_train, y_train)
print("Best parameters:", cv.best_params_)
print("Best accuracy:", cv.best_score_)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__min_samples_split=2, clf__n_estimators=25 ..................
[CV]  clf__min_samples_split=2, clf__n_estimators=25, score=0.24727216843399416, total=  17.9s
[CV] clf__min_samples_split=2, clf__n_estimators=25 ..................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   23.2s remaining:    0.0s


[CV]  clf__min_samples_split=2, clf__n_estimators=25, score=0.2429691101890272, total=  17.9s
[CV] clf__min_samples_split=2, clf__n_estimators=25 ..................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   46.3s remaining:    0.0s


[CV]  clf__min_samples_split=2, clf__n_estimators=25, score=0.2572614107883817, total=  18.2s
[CV] clf__min_samples_split=2, clf__n_estimators=50 ..................


...

[CV]  clf__min_samples_split=3, clf__n_estimators=25, score=0.25265099124020285, total=  16.8s
[CV] clf__min_samples_split=3, clf__n_estimators=50 ..................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  4.0min remaining:    0.0s


[CV]  clf__min_samples_split=3, clf__n_estimators=50, score=0.25019210081450743, total=  25.5s
[CV] clf__min_samples_split=3, clf__n_estimators=50 ..................
[CV]  clf__min_samples_split=3, clf__n_estimators=50, score=0.2517289073305671, total=  24.9s
[CV] clf__min_samples_split=3, clf__n_estimators=50 ..................
[CV]  clf__min_samples_split=3, clf__n_estimators=50, score=0.2529583525434148, total=  25.0s


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  5.6min finished


Best Parameters: {'clf__min_samples_split': 2, 'clf__n_estimators': 50}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
y_pred_cv = cv.predict(X_test)
print(classification_report(y_test.values, y_pred_cv, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.95      0.89      4938
               request       0.88      0.43      0.58      1104
                 offer       0.00      0.00      0.00        26
           aid_related       0.83      0.49      0.61      2714
          medical_help       0.80      0.01      0.01       542
      medical_products       1.00      0.02      0.05       347
     search_and_rescue       0.67      0.01      0.02       192
              security       1.00      0.01      0.01       133
              military       0.00      0.00      0.00       225
           child_alone       0.00      0.00      0.00         0
                 water       0.90      0.15      0.26       430
                  food       0.90      0.25      0.39       731
               shelter       0.91      0.12      0.21       598
              clothing       1.00      0.04      0.08       101
                 money       0.88      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [83]:
def tokenize_adv(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    # Remove stop words
    clean_tokens = [w for w in clean_tokens if w not in stopwords.words("english")]  
    
    return clean_tokens

In [84]:
AdaBoost = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize_adv)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(learning_rate=0.5, n_estimators=100)))
    ])

In [85]:
# Train second model
AdaBoost.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ator=None,
          learning_rate=0.5, n_estimators=100, random_state=None),
           n_jobs=1))])

In [86]:
# Test second model
y_predAdaBoost_test = AdaBoost.predict(X_test)

# classification report on test data
print(classification_report(y_test.values, y_predAdaBoost_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.80      0.97      0.88      4938
               request       0.80      0.50      0.61      1104
                 offer       0.00      0.00      0.00        26
           aid_related       0.78      0.59      0.67      2714
          medical_help       0.65      0.18      0.29       542
      medical_products       0.70      0.25      0.37       347
     search_and_rescue       0.71      0.12      0.21       192
              security       0.33      0.01      0.01       133
              military       0.66      0.23      0.34       225
           child_alone       0.00      0.00      0.00         0
                 water       0.77      0.66      0.71       430
                  food       0.80      0.72      0.76       731
               shelter       0.82      0.51      0.63       598
              clothing       0.76      0.41      0.53       101
                 money       0.62      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

AdaBoost seems to perform better than GridSearch-optimized RandomForestClassifier, therefore the AdaBoostClassifier was selected as model to export.

In [87]:
pickle.dump(AdaBoost, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.